In [44]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow import keras

In [45]:
model = tf.keras.applications.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [46]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [47]:
for layer in model.layers:
  layer.trainable = False

Imagenet을 통해 사전학습된 layer의 가중치를 고정한다

In [48]:
model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [49]:
last_layer = model.get_layer('block5_pool').output

In [60]:
out = tf.keras.layers.Flatten()(last_layer)
out = tf.keras.layers.Dropout(0.4)(out)
out = tf.keras.layers.Dense(2048, activation='relu')(out)
out = tf.keras.layers.Dropout(0.4)(out)
out = tf.keras.layers.Dense(1024, activation='relu')(out)
out = tf.keras.layers.Dense(100)(out)

model = tf.keras.models.Model(model.input, out)
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [61]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar100.load_data()

In [62]:
X_train, X_valid = X_train[:40000], X_train[40000:]
y_train, y_valid = y_train[:40000], y_train[40000:]

X_train.shape, X_valid.shape, X_test.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (10000, 32, 32, 3))

In [63]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
valid_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [64]:
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
valid_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [65]:
def augment(image,label):
  image = tf.image.random_brightness(image, max_delta=0.5) # Random brightness
  image = tf.image.rot90(image, k=1)
  return image, label

def parse_image(images, labels):
  image = tf.cast(images, tf.float32)
  image = tf.image.resize(image, [224, 224])
  image = image / 255.0
  image, labels = augment(image, labels)
  return image, labels

In [66]:
train_dataset = train_dataset.map(parse_image)
valid_dataset = valid_dataset.map(parse_image)
test_dataset = test_dataset.map(parse_image)

In [67]:
model.compile(optimizer = tf.keras.optimizers.Adam(), loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True), metrics = ['accuracy'])

In [68]:
for image, label in train_dataset.take(1):
    print(image.shape)
    print(label.shape)

(64, 224, 224, 3)
(64, 1)


In [ ]:
model.fit(train_dataset, epochs=100, verbose=1, validation_data=(valid_dataset))

Epoch 1/100
103/625 [===>..........................] - ETA: 5:41 - loss: 4.8718 - accuracy: 0.0234